# 04A_Pre_Merging_Baselining_SSA

### This script is updated and developed off of the pybaselineCorrected_merged.ipynb from Beth Galtry. 

## Import modules

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import h5py
import os
import matplotlib as mpl
import pandas as pd        
import math

import sys
!{sys.executable} -m pip install pyFAI
import pyFAI
from pyFAI.gui import jupyter

!{sys.executable} -m pip install pybaselines
from pybaselines import Baseline
from pybaselines.utils import gaussian

pi = math.pi

## Specify diffraction frame to read in

In [ ]:
#beth 
# data file from I11 in form i11-1-61284.nxs, adjust depending on data name and format


data_file_no = "107655"
data_dir = "D:/I11 Beamtime July/RAW_2D/Run_3_X3_0.5VF"  #"C:/path/to/your/data/file/"

data_file = "i11-1-" + data_file_no + ".nxs"

file_path = os.path.join(data_dir, data_file)

In [ ]:
#beth
# creation of folder to store processed information
processing_folder = "04A_Pre_Merging_Baselining/individual_frame_patterns/"
processing_path = os.path.join(data_dir,processing_folder)


if not os.path.exists(processing_path):
    os.makedirs(processing_path)

In [ ]:
#beth
with h5py.File(file_path, 'r') as dat:
    
    #load in as (1, 1441, 1440) shape array
    myData = np.array(dat["/entry1/pixium_hdf/data"][()][:]) 
    
    #extract only (1441,1440)
    diff_frame = myData.reshape(myData.shape[1:])
    
total_intensity = np.sum(diff_frame)

diff_frame

In [ ]:
#beth 
# visualise chosen diffraction frame

plt.rcParams["figure.figsize"]=14,14
plt.title(data_file, fontsize=24)
plt.imshow(diff_frame, cmap='gray', vmin=0, vmax=300)
plt.colorbar()
plt.show()

# Load calibration file

calibrated using pyFAI-calib2 tool

pyFAI: https://pyfai.readthedocs.io/en/v2023.1/

In [ ]:
# load the calibration .poni file 
calibration = pyFAI.load("D:/I11 Beamtime July/calib_fep.poni")   #('C:/poni/file/location/calibration.poni')
calibration

In [ ]:
#load mask for detector frame
oneD_mask = np.load("D:/I11 Beamtime July/calib_fep_mask.npy")    #("C:/mask/file/location/mask.npy")
oneD_mask

## Integration using pyFAI module
#### pyFAI: https://pyfai.readthedocs.io/en/v2023.1/

In [ ]:
threshold_pattern_name = os.path.splitext(os.path.basename(file_path))[0]
threshold_pattern_name = threshold_pattern_name.replace("i11-1-", "") + "_baselineCorrected"

print(threshold_pattern_name)


In [ ]:
#scarlett adaptation

#integrate the 2D pattern into a 1D frame 
oneD_integrated_plot = calibration.integrate1d(
    diff_frame,
    1000,
    unit=pyFAI.units.TTH_DEG,
    radial_range=[0, 30],
    mask=oneD_mask
)

# Plot
jupyter.plot1d(oneD_integrated_plot)
plt.title(threshold_pattern_name, fontsize=15)
plt.xlabel('$2\\theta$ ($^{o}$)')
plt.xlim(4, 30)
#plt.ylim(-0.1, 500)
plt.ylabel('Intensity')
#plt.savefig(processing_path + threshold_pattern_name +"_1D_plot")

# Baseline correction 

### using https://pybaselines.readthedocs.io/en/latest/ mor() baseline correction

In [ ]:

intensity_values = oneD_integrated_plot[1]
two_theta = oneD_integrated_plot[0]
x = two_theta
y = intensity_values

baseline_fitter = Baseline(x_data=x)

In [ ]:
len(oneD_integrated_plot)

In [ ]:
#scarlett
#Determine Half window value :)
# typically PXRD Data uses the range of 5-15 
# You want the baseline to follow the general background trend without cutting into any true diffraction peaks.
plt.figure(figsize=(10,6))
for hw in [3, 7, 10, 15]:
    baseline = baseline_fitter.mor(y, half_window=hw)[0]
    plt.plot(x, baseline, label=f'half_window={hw}')
plt.plot(x, y, 'k--', alpha=0.3, label='data')
plt.legend()
plt.xlabel('2θ (°)')
plt.ylabel('Intensity')
plt.title('Effect of half_window on baseline shape')
plt.show()


In [ ]:
#Scarlett 
#automated estimate of suitable half window value 
# Calculate step size in 2θ
step_size = np.mean(np.diff(x))  # degrees per data point

# Estimate how many data points correspond to ~0.3–0.5° of 2θ background curvature
# (adjust this range if your background varies faster or slower)
background_scale_deg = 0.4  # typical slow-varying background over ~0.4°
half_window_estimate = int(background_scale_deg / step_size / 2)

# Ensure it's an odd, reasonable value (avoid too small or too large)
half_window = max(3, min(half_window_estimate, 25))

print(f"Estimated half_window = {half_window} (based on step size {step_size:.4f}°)")

In [ ]:
# baseline correction, change half_window valuue to best fit dataset

half_window = 6

plt.figure()
plt.plot(x, y, label='data')
plt.plot(x, baseline_fitter.mor(y, half_window=half_window)[0], label=f'half_window={half_window}')
plt.legend()

In [ ]:
# baseline correction
corrected_data = y - baseline_fitter.mor(y, half_window=half_window)[0]

In [ ]:
# plotting of baseline corrected pattern against baseline correction parameters

twoD_pattern_name_baseline_corr = data_file_no +"_baseline_corr_half_window_" + str(half_window)

baseline_corr_name = data_file_no + "half_window_" + str(half_window) + "_baseline"

plt.figure()
plt.plot(x, y, label='data')
plt.plot(x, baseline_fitter.mor(y, half_window=half_window)[0], label=f'half_window={half_window}')
plt.plot(x, corrected_data, label='corrected_baseline')
plt.title(baseline_corr_name, fontsize=15)
plt.xlabel('$2\\theta$ ($^{o}$)')
plt.ylabel('Intensity')
plt.xlim(4, 20)
plt.legend()
plt.savefig(processing_path + twoD_pattern_name_baseline_corr)

In [ ]:
# final pattern

plt.figure()
plt.plot(x, corrected_data, label= 'final_pattern')
plt.title(twoD_pattern_name_baseline_corr, fontsize=15)
plt.xlabel('$2\\theta$ ($^{o}$)')
plt.ylabel('Intensity')
plt.xlim(1, 30)
plt.ylim(-0.1, 900)
plt.savefig(processing_path + twoD_pattern_name_baseline_corr + "_final_pattern")

In [ ]:
# exporting final pattern as .xy data

np.savetxt(processing_path+twoD_pattern_name_baseline_corr+ ".xy", np.c_[x, corrected_data])